# TensorFlow crash course
### **PART 6**

### Autodiff (**TensorFlow’s auto differentiation**)

In [1]:
#  We use Tensorflow's autodiff to compute derivatives

import tensorflow as tf

def func(W1, W2): #  Our simple mathmatical function
    return 3 * W1 ** 2 + 2 * W1 * W2

W1, W2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape: #  Records every operation the variables go through automatically
    z = func(W1, W2)

gradients = tape.gradient(z, [W1, W2]) #  Compute the gradients of the result z based on both variables [w1, w2]
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


### Auto erasing

In [2]:
#  The tape is automatically erased immediately after you call its gradient() method
#  To prevent the above issue we use a tiny magic

with tf.GradientTape(persistent=True) as tape: # Notice that we added the presistent argument
    z = func(W1, W2)

gradients = tape.gradient(z, [W1, W2])
print(gradients)
gradients = tape.gradient(z, [W1, W2]) #  Works fine now
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]
[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


### Autodiff for constants

In [3]:
C1, C2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(C1)
    tape.watch(C2)
    z = func(C1, C2)

gradients = tape.gradient(z, [C1, C2])
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


### Autodiff for vectors

In [4]:
V1, V2 = tf.Variable([1., 3., 5., 7.]), tf.Variable([2., 4., 6., 8.])
with tf.GradientTape() as tape:
    z = func(V1, V2)

jacobians = tape.jacobian(z, [V1, V2])
print(jacobians)

[<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[10.,  0.,  0.,  0.],
       [ 0., 26.,  0.,  0.],
       [ 0.,  0., 42.,  0.],
       [ 0.,  0.,  0., 58.]], dtype=float32)>, <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 2.,  0.,  0.,  0.],
       [ 0.,  6.,  0.,  0.],
       [ 0.,  0., 10.,  0.],
       [ 0.,  0.,  0., 14.]], dtype=float32)>]


### Custom gradient

In [5]:
#  We'll implement a custom gradient for the sinc function
#  In order to do such a thing, we have to use the tf.custom_gradient decorator

@tf.custom_gradient
def sinc(x):
    sin = tf.sin(x)
    cos = tf.cos(x)
    eps = tf.constant(1e-5)
    def sinc_gradient(grad):
        return (x * cos - sin) / (tf.square(x)+eps)
    return sin / (x+eps), sinc_gradient 

x = tf.Variable(1.)
with tf.GradientTape() as tape:
    z = sinc(x)

gradients = tape.gradient(z, x) 
print(gradients)

tf.Tensor(-0.30116567, shape=(), dtype=float32)
